In [10]:

from tg.grammar_ru import Loc
NAME = 'Frei'


CORPUS_FOLDER = Loc.root_path.parent/'corpus_files'
FOLDER = CORPUS_FOLDER/'raw/ficbook'/NAME
MD_FOLDER =CORPUS_FOLDER/'processed/ficbook/'/NAME


In [11]:
from yo_fluq_ds import *


parseable_pages = FileIO.read_json('parseable_pages.json')

In [13]:
import bs4 


def find_single(self,   
            name=None,
            attrs={},
            recursive=True,
            string=None,
            limit=None,
            **kwargs):
    result = self.find_all(name=name, attrs=attrs, recursive=recursive, string=string, limit=limit, **kwargs)
    if len(result)!=1:
        raise ValueError(f'Expected single element, but was {len(result)}')
    return result[0]
bs4.BeautifulSoup.find_single = find_single

def parse(html):
    bs = bs4.BeautifulSoup(html)
    result = Obj()
    result.title = bs.find_single('h1').text
    result.author = ', '.join([c.text for c in bs.find_all('a', attrs={'class':'creator-nickname'})])
    
    result.likes = '-1'
    
    likes = bs.find_all('span', attrs={'class': 'badge-text js-marks-plus'})
    if len(likes) == 1:
        result.likes = likes[0].text

    result.content = bs.find_single('div', attrs={'id': 'content'}).text
    subtitles = bs.find_all('h2')
    if len(subtitles)==1:
        subtitle = subtitles[0]
    elif subtitles[0].text=='18+':
        subtitle = subtitles[1]
    else:
        raise ValueError(f'Unexpected subtitles\n{subtitles}')
    
    
    result.subtitle = subtitle.text
    
    return result

import json

def to_md(parsed):
    result = []
    result.append(f'# {parsed.title}')
    result.append(f'## {parsed.subtitle}')
    result.append(f'$ {json.dumps(dict(author=parsed.author, likes=parsed.likes))}')
    for s in parsed.content.split('\n'):
        s = s.strip()
        if s!='':
            result.append(s)
    return '\n\n'.join(result)



fname = parseable_pages[10]
text = FileIO.read_text(fname)
#print(text)
parsed = parse(FileIO.read_text(fname))
print(to_md(parsed)[:200])

# Два мира - Явь и Сон

## Часть 1

$ {"author": "Ram Ro", "likes": "21"}

На перекрестке тысячи миров.

В пустых глазах давно уже не детских

Опять граничат выдумки из снов

С реальностью холодной, с


In [15]:
os.makedirs(MD_FOLDER, exist_ok=True)
if True:
    for i,p in Query.en(enumerate(parseable_pages)).feed(fluq.with_progress_bar()):
        try:
            parsed = parse(FileIO.read_text(p))
            md = to_md(parsed)
            FileIO.write_text(md,MD_FOLDER/f'{i}.md')
        except:
            print(p)
            raise

0it [00:00, ?it/s]

In [16]:
from tg.grammar_ru.ml.corpus import CorpusBuilder

CorpusBuilder.convert_interformat_folder_to_corpus(
    Loc.corpus_path/'ficbook.base.zip',
    MD_FOLDER.parent,
    ['fandom', 'num']
)

  0%|          | 0/2188 [00:00<?, ?it/s]

In [19]:
from tg.grammar_ru import CorpusReader, Separator

reader = CorpusReader(Loc.corpus_path/'ficbook.base.zip')
toc = reader.get_toc()


In [23]:
toc.groupby('fandom').token_count.sum()

fandom
Frei      1655318
Martin    1584779
Name: token_count, dtype: int64